In [3]:
import sys
pt_models = "../input/pretrainedmodels/pretrained-models.pytorch-master/"
sys.path.insert(0, pt_models)

In [4]:
import glob
import torch
import albumentations
import pandas as pd
import numpy as np
from tqdm import tqdm
import joblib
from PIL import Image
import torch.nn as nn
from torch.nn import functional as F


In [5]:
TRAIN_BATCH_SIZE=32
TEST_BATCH_SIZE=16
MODEL_MEAN=(0.4, 0.4, 0.4)
MODEL_STD=(0.2,0.2,0.2)
IMG_HEIGHT = 137
IMG_WIDTH = 236
DEVICE="cuda"


In [6]:
class Resnet34(nn.Module):
    def __init__(self, pretrained=True):
        super(Resnet34, self).__init__()
        if pretrained:
            self.model = pretrainedmodels.__dict__['resnet34'](pretrained='imagenet')
        else:
            self.model = pretrainedmodels.__dict_['resnet34'](pretrained=None)
        
        self.l0 = nn.Linear(512, 168)
        self.l1 = nn.Linear(512, 11)
        self.l2 = nn.Linear(512, 7)
    
    def forward(self, x):
        bs, _, _, _  = x.shape
        x = self.model.features(x)
        x = F.adaptive_avg_pool2d(x, 1).reshape(bs, -1)
        l0 = self.l0(x)
        l1 = self.l1(x)
        l2 = self.l2(x)

        return l0, l1, l2

In [7]:
class BengaliaiDataset:
    def __init__(self, df, img_height, img_width, mean, std):
        self.image_ids = df.image_id.values
        self.img_arr = df.loc[:, 1:].values
        self.aug = albumentations.Compose([
            albumentations.Resize(img_height, img_width, always_apply=True),
            albumentations.Normalize(mean, std, always_apply=True)
        ])


    def __len__(self):
        return len(self.image_ids)
    

    def __getitem__(self, item):
        image = self.img_arr[item,:]
        img_id = self.image_ids[item]
        image = image.reshape(137, 236).astype(float)
        image = Image.fromarray(image).convert("RGB")

        image = self.aug(image=np.array(image))["image"]
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        return {
            "image" : torch.tensor(image, dtype = torch.float),
            "img_id" : img_id
        }

In [ ]:
model = Resnet34(pretrained=False)
model.load_state_dict(torch.load("../input/bengaliaimodels/resnet34_fold0.bin"))
model.eval()

In [ ]:
predictions = []
for file_idx in range(4):
    df = pd.read_csv("../input/bengaliai-cv19/test_image_data_{}.parquet".format(file_idx))
    test_ds = BengaliaiDataset(df = df, img_height=IMG_HEIGHT, img_width=IMG_WIDTH, mean = MODEL_MEAN, std = MODEL_STD)
    test_loader = torch.utils.data.DataLoader(dataset=test_ds, batch_size=TEST_BATCH_SIZE, shuffle=False)
    
    
    for bi, d in enumerate(test_ds);
        image = d["image"]
        img_id = d["img_id"]
        image = image.to(DEVICE, dtype = torch.float)
        g, v, c = model(image)
        
        g = np.argmax(g, axis=1)
        v = np.argmax(v, axis=1)
        c = np.argmax(c, axis=1)
        
        for ii, iimd in enumerate(img_id);
            predictions.append((iimd, g[ii], v[ii], c[ii]))
            
        